# `libiio`: library for interfacing with IIO devices

XADC is currently the only IIO device on Red Pitaya.
XADC provides measurements of Zynq chip temperature, supply voltages and a set of analog inputs.
The set of measured analog inputs is defined by a device tree node.

The kernel IIO driver provides a `sysfs` interface for accessing XADC measurements.
There are actually two drivers running at the same time.

## `compatible = "xlnx,zynq-xadc-1.00.a";`

This driver is accessing XADC over JTAG, and provides only
temperature and supply voltage measurements, at a slow data rate.
The advantage is, it does not require FPGA code and is therefore always available.
All data is accessed over `sysfs`.

## `compatible = "xlnx,axi-xadc-1.00.a";`

This driver requires FPGA code connecting the XADC to the CPU over the AXI bus.
The full feature set is provided.
External voltage measurements are available with the full data rate.
In addition to the `sysfs` interface a cyclic buffer can be obtained from a device node.

The next code is using Python bindings to `libiio` list the functionality exposed over `sysfs`.

In [ ]:
import iio

def main():
    ctx = iio.Context()

    print ('Library version: {}.{} (git tag: {})'.format(iio.version[0], iio.version[1], iio.version[2].decode()))

    print ('IIO context created: %s' % ctx.name)
    print ('Backend version: {}.{} (git tag: {})'.format(ctx.version[0], ctx.version[1], ctx.version[2].decode()))
    print ('Backend description string: {}'.format(ctx.description.decode()))

    print ('IIO context has %u devices:' % len(ctx.devices))

    for dev in ctx.devices:
        print ('\t{}: {}'.format(dev.id.decode(), dev.name.decode()))

        if dev is iio.Trigger:
            print ('Found trigger! Rate: {} Hz'.format(dev.frequency))

        print ('\t\t{} channels found:'.format(len(dev.channels)))

        for chn in dev.channels:
            print ('\t\t\t{}: {} ({})'.format(chn.id.decode(), chn.name.decode() if chn.name else "", 'output' if chn.output else 'input'))

            if len(chn.attrs) != 0:
                print ('\t\t\t{} channel-specific attributes found:'.format(len(chn.attrs)))

            for attr in chn.attrs:
                print ('\t\t\t\t{}, value: {}'.format(attr.decode(), chn.attrs[attr].value.decode()))

        if len(dev.attrs) != 0:
            print ('\t\t{} device-specific attributes found:'.format(len(dev.attrs)))

        for attr in dev.attrs:
            print ('\t\t\t{}, value: '.format(attr.decode(), dev.attrs[attr].value.decode()))

        if len(dev.debug_attrs) != 0:
            print ('\t\t{} debug attributes found:'.format(len(dev.debug_attrs)))

        for attr in dev.debug_attrs:
            print ('\t\t\t{}, value: '.format(attr.decode(), dev.debug_attrs[attr].value.decode()))

if __name__ == '__main__':
    main()

The next example is reading measurements provided by the `zynq-xadc` driver as `sysfs` files.
Based on my experience an undocumented ratio of 1000 has to be used to achieve correct results.

Values `raw`, `offset` and `scale` are used to calculate the temperature.

$$ T = (raw + offset) \cdot scale $$

Values `raw` and `scale` are used to calculate supply voltages.

$$ U = raw \cdot scale $$

In [ ]:
import iio

ctx = iio.Context()
dev = ctx.devices[2]

for chn in dev.channels:
    print ('{}: {} ({})'.format(chn.id.decode(), chn.name.decode() if chn.name else "", 'output' if chn.output else 'input'))
    raw    = chn.attrs[b'raw'].value.decode()
    scale  = chn.attrs[b'scale'].value.decode()
    if len(chn.attrs) == 3:
        offset = chn.attrs[b'offset'].value.decode()
        print ('\t({}+{})*{}/1000={}'.format(raw, offset, scale, (int(raw)+int(offset))*float(scale)/1000))
    else:
        print ('\t{}*{}/1000={}'.format(raw, scale, int(raw)*float(scale)/1000))

The next example is reading measurements provided by the `axi-xadc` driver as `sysfs` files.
Based on my experience an undocumented ratio of 1000 has to be used to achieve correct results.

The equations are the same as in the previous example.
This interface also provides slow analog input pin voltages `vaux0`, `vaux1`, `vaux8` and `vaux9`.

In [ ]:
import iio

ctx = iio.Context()
dev = ctx.devices[3]

for chn in dev.channels:
    print ('{}: {} ({})'.format(chn.id.decode(), chn.name.decode() if chn.name else "", 'output' if chn.output else 'input'))
    raw    = chn.attrs[b'raw'].value.decode()
    scale  = chn.attrs[b'scale'].value.decode()
    if len(chn.attrs) == 6:
        offset = chn.attrs[b'offset'].value.decode()
        print ('\t({}+{})*{}/1000={}'.format(raw, offset, scale, (int(raw)+int(offset))*float(scale)/1000))
    else:
        print ('\t{}*{}/1000={}'.format(raw, scale, int(raw)*float(scale)/1000))

The following code is an attempt to get a buffer containing XADC samples.
For now it crashes the Linux kernel, requiering a reboot.

I can get it to work in CLI using C example:
```
/home/jupyter/libiio/examples/dummy-iiostream -d xadc -t xadc0-samplerate -b 64 -c 2
```
But I have to comment out `zynq-xadc` in the device tree,
since both have the same device name (kernel driver name).

In [ ]:
import numpy as np

data_size = 64

import iio
ctx = iio.Context()
trg = ctx.devices[1]
dev = ctx.devices[3]
# due to a probable bug in the axi-xadc driver all channels have to be enabled,
# otherwise the driver crashes the kernel
for chn in dev.channels:
    chn.enabled = True
dev.trigger = trg
buf = iio.Buffer(dev, data_size, cyclic=False)

print ([dev.channels[i].id.decode() for i in range(len(dev.channels))])
print ([dev.channels[i].scan_element for i in range(len(dev.channels))])
print ([dev.channels[i].enabled for i in range(len(dev.channels))])

In [ ]:
cnt = buf.refill()

In [ ]:
data_buf = buf.read()
# data = np.recarray(data_size, data_type, buf=data_buf)
data = np.frombuffer(data_buf, np.dtype('uint16'))
# reshape array and transpose
data = data.reshape((data_size, len(dev.channels))).T

In [ ]:
[x>>3 for x in data]

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py
import plotly.graph_objs as go

init_notebook_mode(connected=True)

data_x = np.linspace(0, 1, data_size)

# temperature trace
chn = dev.channels[0]
scale  = chn.attrs[b'scale'].value.decode()
offset = chn.attrs[b'offset'].value.decode()
temp0 = go.Scatter(
    x = data_x,
    y = ((data[0]>>4) + int(offset)) * float(scale) / 1000,
    mode = 'lines',
    name = 'temp0'
)

# voltage traces
traces = []
for chn in dev.channels:
    if len(chn.attrs) == 5:
        scale  = chn.attrs[b'scale'].value.decode()
        traces.append(go.Scatter(
            x = data_x,
            y = (data[0]>>4) * float(scale) / 1000,
            mode = 'lines',
            name = chn.name.decode()
        ))

# Plot and embed in ipython notebook!
iplot([temp0], show_link=False)
iplot(traces, show_link=False)